# Connectedness

In [1]:
from algopy import graph, graphmat
from algopy.graph import Graph
from algopy.graphmat import GraphMat

Let's first recall how we classify connected components of a graph. Output is a list of size order of the graph, where at a vertex index one finds the corresponding connected component index.

In [2]:
def __cc(G, s, P, nb):
    P[s] = nb
    for adj in G.adjlists[s]:
        if P[adj] == 0:
            __cc(G, adj, P, nb)

def cc(G):
    """Computes number of connected components and indexes them.
    
    Arguments:
        G (Graph): undirected graph.
    
    Returns:
        list: mapping list for vertex belonging to given components.
        int : number of components
        
    """
    P = [0] * G.order
    nb = 0
    for i in range(G.order):
        if P[i] == 0:
            nb += 1
            __cc(G, i, P, nb)
    return P, nb

## Union - Find

We build up a covering tree out of a list of links in the graph. We optimize in order to get trees having small depths for output.

In [3]:
def find(x, p):
    """Searches for the representant of connected component of x in p."""
    rx = x
    while p[rx] > -1:
        rx = p[rx]
    return rx

def union(x, y, p):
    """Links representants of connected components of x and y if different."""
    rx = find(x, p)
    ry = find(y, p)
    if ry != rx:
        if p[rx] > p[ry]:
            p[ry] = p[rx] + p[ry]
            p[rx] = ry
        else:
            p[rx] = p[rx] + p[ry]
            p[ry] = rx
    
def union_find(L, n):
    """Builds covering forest out of links between vertices."""
    p = [-1]*n
    for (x, y) in L:
        union(x, y, p)
    return p

## KrisNet

In [4]:
def krisnet_solver(L, n):
    """Returns chain to add to the structure to be connected."""
    cc = union_find(L, n)
    chain = []
    for i in range(n):
        if cc[i] < 0:
            chain.append(i)
    return chain

## Journey to The Moon

In [5]:
def journey(L, n):
    """Number of possibilities of choosing 2 astronauts from 2 distinct countries"""
    cc = union_find(L, n)
    res = 0
    for i in range(n):
        if cc[i] < 0:
            for j in range(i+1, n):
                if cc[j] < 0:
                    res += cc[i]*cc[j]
    return res

## Connected Components from Edges

First a version that returns the vector storing a covering forest.

In [6]:
def find2(x, p):
    """Searches for the representant of connected component of x in p."""
    rx = x
    while p[rx] > -1:
        rx = p[rx]
    return rx

def union2(x, y, p):
    """Links representants of connected components of x and y if different."""
    rx = find2(x, p)
    ry = find2(y, p)
    if ry != rx:
        if p[rx] > p[ry]:
            p[ry] = p[rx] + p[ry]
            p[rx] = ry
        else:
            p[rx] = p[rx] + p[ry]
            p[ry] = rx
        return -1
    else:
        return 0
    
def union_find2(L, n):
    """Builds covering forest out of links between vertices."""
    p = [-1]*n
    k = n
    for (x, y) in L:
        k += union2(x, y, p)
    return k, p

A second version giving back the vector of components ; they are indexed from $1$ to the number of connected components, for a given vertex one finds corresponding component index at vertex index.

In [7]:
def cc_from_edges(L, n):
    """Computing vector of connected components from list of links."""
    p = union_find(L, n)
    cc = [None]*n
    k = 0
    for s in range(n):
        if p[s] < 0:
            k += 1
            cc[s] = k
    for s in range(n):
        cc[s] = cc[find(s, p)]
    return (cc, k)

A more optimized version which gives back a vector enabling connected component identification but that is not indexed from $1$ to the number of connected components. 

In [8]:
def __union(x, y, cc):
    if cc[x] != cc[y]:
        if cc[x] < cc[y]:
            mini, maxi = cc[x], cc[y]
        else:
            mini, maxi = cc[y], cc[x]
        for c_index in range(n):
            if cc[c_index] == maxi:
                cc[c_index] = mini
        return -1
    return 0
    
def cc_from_edges(L, n):
    """Builds connected component vector out of links between vertices."""
    cc = [i+1 for i in range(n)]
    k = n
    for (x, y) in L:
        k += __union(x, y, cc)
    return k, cc

## Connected Components from Warshall

In [9]:
def cc_from_warshall(M):
    """Computes connected components vector from Warshall."""
    n = len(M)
    cc = [0] * n
    k = 0
    for x in range(n):
        if cc[x] == 0:
            k += 1
            cc[x] = k
            for y in range(x+1, n):
                if M[x][y]:
                    cc[y] = k
    return k, cc

## Warshall

In [2]:
def init_matrix(n, m, val):
    """Initializes matrix of size n and m filling it with values val."""
    M = []
    for i in range(n):
        M.append([])
        for j in range(m):
            M[i].append(val)
    return M

@Nathalie's versions.

In [3]:
def warshall(G):
    M = init_matrix(G.order, G.order, 0)
    for s in range(G.order):
        for adj in G.adjlists[s]:
            M[s][adj] = 1
            
    for k in range(G.order):
        for i in range(G.order):
            for j in range(G.order):
                M[i][j] = M[i][j] or M[i][k] and M[k][j]
    
    return M

def warshall_quick(G):
    M = init_matrix(G.order, G.order, 0)
    for s in range(G.order):
        for adj in G.adjlists[s]:
            M[s][adj] = 1
            
    for k in range(G.order):
        for i in range(G.order):
            if M[i][k]:
                for j in range(G.order):
                    M[i][j] = M[i][j] or M[k][j]
    

## I Want To Be Tree

In [10]:
def __betree(G, s, P, nocc, cc):
    for i in range(G.order):
        if G.adj[s][i]:
            if P[i] is None:
                P[i] = s
                cc[i] = nocc
                __betree(G, i, P)
            elif P[s] != i:
                G.adj[s][i] -= 1
                G.adj[i][s] -= 1
def betree(G):
    """Transform a graph into a tree."""
    P = [None] * G.order
    cc = [0] * G.order
    nocc = 1
    __betree(G, 0, P, nocc, cc)
    for i in range(1, G.order):
        nocc += 1
        if P[i] is None:
            __betree(G, i, P, nocc, cc)
            G.addedge(0, i)
    return G, P

## Even Tree

In [4]:
def __even_tree(G, src, M):
    M[src] = True
    size = 1
    removed = 0
    for succ in G.adjLists[src]:
        if not M[succ]:
            s, r = __even_tree(G, succ, M)
            removed += r
            if s % 2 == 0:
                removed += 1
            else:
                size += s
    return size, removed

def even_tree(G):
    M = [False]*G.order
    (_, removed) = dfs(G, 0, M)
    return removed

## Strongly Connected Components -- Naive

In [11]:
def reverse(G):
    G_inv = Graph(G.order, True)
    for s in range(G.order):
        for adj in G.adjlists[s]:
            G_inv.addedge(adj, s)
    return G_inv

In [1]:
def __dfs(G, s, M):
    M[s] = True
    for adj in G.adjlists[s]:
        if M[adj] is None:
            __dfs(G, adj, M)
        
def dfs(G, s):
    """Single traversal starting at vertex s."""
    M = [None]*G.order
    __dfs(G, s, M)
    return M

def scc_naive(G):
    G_inv = reverse(G)
    scc = [0]*G.order
    no_scc = 1
    for s in range(G.order):
        if not scc[s]:
            plus = dfs(G, s)
            minus = dfs(G_inv, s)
            for vertex in range(G.order):
                if plus[vertex] and minus[vertex]:
                    scc[vertex] = no_scc
            no_scc += 1
    return scc, no_scc

## Kosaraju algorithm

This one has been already given by Nathalie on her webpage. 

## Tarjan Algorithm

In [13]:
from algopy import stack
from algopy.stack import Stack

In [14]:
stack??

In [15]:
def __tarjan(G, x, pref, cpt, scc, no_scc, v_stack):
    # Preorder treatments
    cpt += 1
    pref[x] = cpt
    v_stack.push(x)
    return_x = pref[x]
    # Recursive bloc
    for adj in G.adjlists[x]:
        if pref[adj] == 0:
            (return_adj, cpt, no_scc) = __tarjan(G, adj, pref, cpt, scc, no_scc, v_stack)
            return_x = min(return_x, return_adj)
        else:
            return_x = min(return_x, pref[adj])
    # Postorder treatment
    if pref[x] == return_x:
        no_scc += 1
        y = -1
        while y != x:
            y = v_stack.pop()
            scc[y] = no_scc
            pref[y] = G.order
    return return_x, cpt, no_scc
    
def tarjan(G):
    """Computes vector of SCC using Tarjan algorithm"""
    pref, scc = [0]*G.order, [0]*G.order
    cpt, no_scc = 0, 0
    v_stack = Stack()
    for s in range(G.order):
        if pref[s] == 0:
            (_, cpt, no_scc) = __tarjan(G, s, pref, cpt, scc, no_scc, v_stack)
    return no_scc, scc